<a href="https://colab.research.google.com/github/santhipsengottuvel/Insurance_Cross_Selling-Project_using_RAPIDS/blob/main/Insurance_Cross_Selling_Project_using_RAPIDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Insurance cross selling project using Rapids

## Verifying the GPU Specifications

In [ ]:
# !nvidia-smi

## Installing Rapids

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py


fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Installing RAPIDS remaining 24.4.* libraries
Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com

        ***********************************************************************
        The pip install of RAPIDS is complete.
        
        Please do not run any further installation from the conda based installation methods, as they may cause issues!
        
        Please ensure that you're pulling from the git repo to remain updated with the latest working install scripts.

        Troubleshooting:
            - If there is an installation failure, please check back on RAPIDSAI owned templates/notebooks to see how to update your personal files. 
            - If an installation failure persists when using the latest script, please make an issue on https://github.com/rapidsai-community/rapidsai-csp-utils
        *****************************************************************

## Verifying that Rapids had been installed properly by checking its versions

In [ ]:
import cudf
cudf.__version__

'24.04.01'

In [ ]:
import cuml
cuml.__version__

'24.04.00'

In [ ]:
import cugraph
cugraph.__version__

'24.04.00'

In [ ]:
import cuspatial
cuspatial.__version__

'24.04.00'

In [ ]:
import cuxfilter
cuxfilter.__version__

'24.04.01'

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

## Reading Dataset

In [ ]:
df=pd.read_csv("/content/train.csv")

In [ ]:
df.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187.0,0.0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288.0,1.0
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254.0,0.0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76.0,0.0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294.0,0.0


In [ ]:
df.shape

(3097213, 12)

In [ ]:
Vehicle_age=pd.DataFrame(df['Vehicle_Age'].value_counts())

In [ ]:
Vehicle_age

,count
Vehicle_Age,
1-2 Year,1611938
< 1 Year,1357048
> 2 Years,128226
> 2,1


## Feature Engineering

In [ ]:
df.isnull().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          1
Annual_Premium          1
Policy_Sales_Channel    1
Vintage                 1
Response                1
dtype: int64

In [ ]:
df.drop('id',axis=1,inplace=True)

In [ ]:
df.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187.0,0.0
1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288.0,1.0
2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254.0,0.0
3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76.0,0.0
4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294.0,0.0


In [ ]:
X=df.iloc[:,:10]
Y=df['Response']

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [ ]:
X_test['Vehicle_Age'].isnull().sum()

0

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
Ordinal=OrdinalEncoder()
X_train['Vehicle_Age']=Ordinal.fit_transform(X_train[['Vehicle_Age']])
X_test['Vehicle_Age']=Ordinal.transform(X_test[['Vehicle_Age']])

In [ ]:
X_train.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
2306828,Male,48,1,29.0,0,0.0,Yes,26324.0,26.0,74.0
497036,Female,20,1,25.0,0,1.0,No,2630.0,160.0,139.0
893329,Female,41,1,3.0,0,0.0,Yes,24206.0,154.0,81.0
2279319,Female,42,1,28.0,1,0.0,No,41615.0,11.0,278.0
1344052,Male,57,1,36.0,1,0.0,No,33911.0,152.0,287.0


In [ ]:
X_train['Vehicle_Age'].value_counts(),X_train["Gender"].value_counts(),X_train["Region_Code"].value_counts()

(Vehicle_Age
 0.0    1209626
 1.0    1017367
 3.0      95915
 2.0          1
 Name: count, dtype: int64,
 Gender
 Male      1257807
 Female    1065102
 Name: count, dtype: int64,
 Region_Code
 28.0    697379
 8.0     206369
 46.0    116516
 41.0    112271
 15.0     81433
 30.0     74145
 29.0     68075
 50.0     60954
 11.0     55915
 36.0     52976
 3.0      49566
 33.0     47053
 47.0     46073
 35.0     40619
 6.0      36910
 45.0     32205
 37.0     31871
 18.0     29680
 39.0     27882
 14.0     27130
 21.0     25622
 10.0     25462
 2.0      23945
 48.0     22960
 13.0     21908
 9.0      18900
 12.0     18686
 7.0      18595
 32.0     16009
 27.0     15945
 43.0     15473
 17.0     14965
 26.0     14344
 25.0     14160
 24.0     14144
 38.0     12214
 20.0     11980
 31.0     11840
 0.0      11767
 16.0     11152
 23.0     11053
 4.0      10607
 49.0     10243
 34.0      9686
 19.0      9055
 22.0      7542
 5.0       7542
 40.0      7259
 1.0       6846
 44.0      4170
 42.0   

In [ ]:
round(y_train.value_counts()/y_train.shape[0]*100 , 2)

Response
0.0    87.72
1.0    12.28
Name: count, dtype: float64

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2322909 entries, 2306828 to 2219110
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Gender                object 
 1   Age                   int64  
 2   Driving_License       int64  
 3   Region_Code           float64
 4   Previously_Insured    int64  
 5   Vehicle_Age           float64
 6   Vehicle_Damage        int64  
 7   Annual_Premium        float64
 8   Policy_Sales_Channel  float64
 9   Vintage               float64
dtypes: float64(5), int64(4), object(1)
memory usage: 194.9+ MB


### Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
label1=LabelEncoder()
label2=LabelEncoder()

X_train['Vehicle_Damage']=label2.fit_transform(X_train[['Vehicle_Damage']])
X_test['Vehicle_Damage']=label2.transform(X_test[['Vehicle_Damage']])

In [ ]:
X_train.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
2306828,Male,48,1,29.0,0,0.0,1,26324.0,26.0,74.0
497036,Female,20,1,25.0,0,1.0,0,2630.0,160.0,139.0
893329,Female,41,1,3.0,0,0.0,1,24206.0,154.0,81.0
2279319,Female,42,1,28.0,1,0.0,0,41615.0,11.0,278.0
1344052,Male,57,1,36.0,1,0.0,0,33911.0,152.0,287.0


In [ ]:
X_train['Gender']=label1.fit_transform(X_train[['Gender']])
X_test['Gender']=label1.transform(X_test[['Gender']])

In [ ]:
X_train.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
2306828,1,48,1,29.0,0,0.0,1,26324.0,26.0,74.0
497036,0,20,1,25.0,0,1.0,0,2630.0,160.0,139.0
893329,0,41,1,3.0,0,0.0,1,24206.0,154.0,81.0
2279319,0,42,1,28.0,1,0.0,0,41615.0,11.0,278.0
1344052,1,57,1,36.0,1,0.0,0,33911.0,152.0,287.0


In [ ]:
X_train.corr()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
Gender,1.000000,0.157539,-0.019648,0.000552,-0.088025,-0.083895,0.097456,0.009858,-0.115561,-0.008986
Age,0.157539,1.000000,-0.078156,0.037666,-0.276434,-0.379883,0.288231,0.056770,-0.592102,-0.013897
Driving_License,-0.019648,-0.078156,1.000000,-0.000951,0.013871,0.021286,-0.015781,-0.007650,0.042649,-0.001301
Region_Code,0.000552,0.037666,-0.000951,1.000000,-0.022158,-0.015925,0.026385,-0.002079,-0.037198,-0.006320
Previously_Insured,-0.088025,-0.276434,0.013871,-0.022158,1.000000,0.103713,-0.836322,0.007352,0.237469,0.019901
Vehicle_Age,-0.083895,-0.379883,0.021286,-0.015925,0.103713,1.000000,-0.098366,0.039445,0.282278,0.010836
Vehicle_Damage,0.097456,0.288231,-0.015781,0.026385,-0.836322,-0.098366,1.000000,0.012247,-0.241163,-0.020018
Annual_Premium,0.009858,0.056770,-0.007650,-0.002079,0.007352,0.039445,0.012247,1.000000,-0.129085,0.003018
Policy_Sales_Channel,-0.115561,-0.592102,0.042649,-0.037198,0.237469,0.282278,-0.241163,-0.129085,1.000000,0.009760
Vintage,-0.008986,-0.013897,-0.001301,-0.006320,0.019901,0.010836,-0.020018,0.003018,0.009760,1.000000


In [ ]:
y_train.value_counts()

Response
0.0    2037661
1.0     285247
Name: count, dtype: int64

In [ ]:
df_concat=pd.concat([X_train,y_train],axis=1)

In [ ]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2322909 entries, 2306828 to 2219110
Data columns (total 11 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Gender                int64  
 1   Age                   int64  
 2   Driving_License       int64  
 3   Region_Code           float64
 4   Previously_Insured    int64  
 5   Vehicle_Age           float64
 6   Vehicle_Damage        int64  
 7   Annual_Premium        float64
 8   Policy_Sales_Channel  float64
 9   Vintage               float64
 10  Response              float64
dtypes: float64(6), int64(5)
memory usage: 212.7 MB


In [ ]:
df_concat.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
2306828,1,48,1,29.0,0,0.0,1,26324.0,26.0,74.0,1.0
497036,0,20,1,25.0,0,1.0,0,2630.0,160.0,139.0,0.0
893329,0,41,1,3.0,0,0.0,1,24206.0,154.0,81.0,0.0
2279319,0,42,1,28.0,1,0.0,0,41615.0,11.0,278.0,0.0
1344052,1,57,1,36.0,1,0.0,0,33911.0,152.0,287.0,0.0


In [ ]:
majority_response=df_concat[df_concat['Response']==0]
minority_response=df_concat[df_concat['Response']==1]

In [ ]:
print(majority_response.shape)
print(minority_response.shape)

(2037661, 11)
(285247, 11)


In [ ]:
sam=len(majority_response)//2
majority_response_sampled=majority_response.sample(sam,random_state=40)

In [ ]:
majority_response_sampled.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
1468060,1,41,1,15.0,0,0.0,1,2630.0,124.0,138.0,0.0
2864312,0,23,1,10.0,1,1.0,0,30791.0,152.0,93.0,0.0
3069387,1,44,1,33.0,0,0.0,1,30537.0,26.0,173.0,0.0
524951,1,39,1,28.0,0,0.0,1,40181.0,122.0,102.0,0.0
1007833,0,27,1,27.0,1,1.0,0,2630.0,152.0,125.0,0.0


In [ ]:
majority_response_sampled.shape

(1018830, 11)

In [ ]:
df_sampled=pd.concat([majority_response_sampled,minority_response])

In [ ]:
df_sampled.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
1468060,1,41,1,15.0,0,0.0,1,2630.0,124.0,138.0,0.0
2864312,0,23,1,10.0,1,1.0,0,30791.0,152.0,93.0,0.0
3069387,1,44,1,33.0,0,0.0,1,30537.0,26.0,173.0,0.0
524951,1,39,1,28.0,0,0.0,1,40181.0,122.0,102.0,0.0
1007833,0,27,1,27.0,1,1.0,0,2630.0,152.0,125.0,0.0


In [ ]:
df_sampled.shape

(1304077, 11)

In [ ]:
df_sampled['Response'].value_counts()

Response
0.0    1018830
1.0     285247
Name: count, dtype: int64

In [ ]:
round(len(df_sampled[df_sampled['Response']==0])/len(df_sampled['Response'])*100)

78

In [ ]:
df_sampled.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
1468060,1,41,1,15.0,0,0.0,1,2630.0,124.0,138.0,0.0
2864312,0,23,1,10.0,1,1.0,0,30791.0,152.0,93.0,0.0
3069387,1,44,1,33.0,0,0.0,1,30537.0,26.0,173.0,0.0
524951,1,39,1,28.0,0,0.0,1,40181.0,122.0,102.0,0.0
1007833,0,27,1,27.0,1,1.0,0,2630.0,152.0,125.0,0.0


In [ ]:
X_Sam=df_sampled.drop(columns='Response',axis=1)
Y_Sam=df_sampled["Response"]

In [ ]:
X_Sam.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
1468060,1,41,1,15.0,0,0.0,1,2630.0,124.0,138.0
2864312,0,23,1,10.0,1,1.0,0,30791.0,152.0,93.0
3069387,1,44,1,33.0,0,0.0,1,30537.0,26.0,173.0
524951,1,39,1,28.0,0,0.0,1,40181.0,122.0,102.0
1007833,0,27,1,27.0,1,1.0,0,2630.0,152.0,125.0


In [ ]:
from sklearn.model_selection import train_test_split
x_train_sam,x_test_sam,y_train_sam,y_test_sam=train_test_split(X_Sam,Y_Sam,random_state=42)

### Over Sampling

In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter
count1=Counter(y_train_sam)
smote=SMOTE(random_state=42)
x_train_sm,y_train_sm=smote.fit_resample(x_train_sam,y_train_sam)
count2=Counter(y_train_sm)

In [ ]:
print("Before Sampling:",count1)
print("After Sampling:,",count2)

Before Sampling: Counter({0.0: 764324, 1.0: 213733})
After Sampling:, Counter({0.0: 764324, 1.0: 764324})


In [ ]:
df_concat_final=pd.concat([x_train_sm,y_train_sm],axis=1)

In [ ]:
df_concat_final.head()

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,21,1,14.0,1,1.0,0,25404.0,152.0,142.0,0.0
1,1,39,1,11.0,1,0.0,0,2630.0,124.0,122.0,0.0
2,0,22,1,14.0,1,1.0,0,2630.0,152.0,49.0,0.0
3,0,21,1,35.0,0,1.0,1,27430.0,160.0,140.0,0.0
4,0,71,1,28.0,0,0.0,0,2630.0,23.0,113.0,0.0


In [ ]:
df_concat_final['Response'].value_counts()

Response
0.0    764324
1.0    764324
Name: count, dtype: int64

## Model Building using Rapids

In [ ]:
import cudf,cuml ## importing cudf and cuml

In [ ]:
## Changing pandas dataframe to cudf dataframe
new_df=cudf.from_pandas(df_concat_final)

In [ ]:
type(new_df)

cudf.core.dataframe.DataFrame

In [ ]:
from cuml.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(new_df.iloc[:,:-1],new_df.iloc[:,-1],test_size=0.20,random_state=42)

In [ ]:
## importing the libraries using cuml instead of scikit learn
from cuml.ensemble import RandomForestClassifier
from cuml.metrics import accuracy_score
from cuml.metrics import confusion_matrix

In [ ]:
rf=RandomForestClassifier()

In [ ]:
rf.fit(x_train,y_train)

RandomForestClassifier()

In [ ]:
y_pred=rf.predict(x_test)

In [ ]:
print(accuracy_score(y_test,y_pred))


0.8256494998931885


In [ ]:
from cuml.linear_model import LogisticRegression
Logistic=LogisticRegression()
Logistic.fit(x_train,y_train)

LogisticRegression()

In [ ]:
y_pred_logistic=Logistic.predict(x_test)

In [ ]:
print(accuracy_score(y_test,y_pred_logistic))

0.7763280272483826
